In [4]:
import numpy as np
import pandas as pd
import os

In [5]:
data_dir = 'vehicles.csv'

In [6]:
data = pd.read_csv( data_dir)


### Preprocessing

In [1]:
from sklearn.model_selection import train_test_split

In [8]:
data_train, data_test = train_test_split(data, test_size=0.2, random_state=42)

In [14]:
data_train

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
366318,7310804917,https://austin.craigslist.org/ctd/d/houston-20...,austin,https://austin.craigslist.org,46995,2017.0,ford,f-150,NaN,8 cylinders,...,NaN,truck,black,https://images.craigslist.org/00202_fA8s718RZU...,This 2017 Ford F-150 Lariat 4WD SuperCrew 5.5'...,NaN,tx,29.678380,-95.435120,2021-04-22T09:03:58-0500
56271,7315327041,https://sandiego.craigslist.org/nsd/ctd/d/palm...,san diego,https://sandiego.craigslist.org,0,2018.0,acura,mdx,NaN,6 cylinders,...,NaN,SUV,NaN,https://images.craigslist.org/00Q0Q_b7u8uPQBQE...,Coachella Valley Volkswagen STOCK #: P...,NaN,ca,33.755638,-116.286258,2021-05-01T08:55:38-0700
264620,7310280162,https://albany.craigslist.org/ctd/d/new-lebano...,albany,https://albany.craigslist.org,36900,2016.0,ford,f-250 super duty,excellent,8 cylinders,...,full-size,truck,white,https://images.craigslist.org/00c0c_jcqfcYXhpQ...,2016 Ford F-250 Super Duty XL 4x4 4dr Crew Cab...,NaN,ny,42.475900,-73.377300,2021-04-21T09:39:24-0400
88787,7315777281,https://washingtondc.craigslist.org/nva/ctd/d/...,"washington, DC",https://washingtondc.craigslist.org,1399,2021.0,NaN,WOLF ISLANDER,NaN,NaN,...,NaN,other,orange,https://images.craigslist.org/00000_af225n2gi4...,2021 WOLF ISLANDER 50CC Offered by: Beyond...,NaN,dc,38.759303,-77.454234,2021-05-02T12:05:03-0400
341412,7306277173,https://york.craigslist.org/ctd/d/york-2018-bu...,york,https://york.craigslist.org,29590,2018.0,buick,enclave essence sport,good,6 cylinders,...,NaN,SUV,white,https://images.craigslist.org/00w0w_36PXCHco1K...,Carvana is the safer way to buy a car During t...,NaN,pa,39.960000,-76.730000,2021-04-13T14:21:01-0400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,7316779292,https://albuquerque.craigslist.org/ctd/d/albuq...,albuquerque,https://albuquerque.craigslist.org,6800,2013.0,volkswagen,jetta se,good,5 cylinders,...,mid-size,sedan,silver,https://images.craigslist.org/01414_8DFTgDUqfH...,"2013 VW Jetta SE 2.5L 5-speed, A/C, pwr. windo...",NaN,nm,35.195300,-106.609300,2021-05-04T10:37:26-0600
365838,7311855271,https://austin.craigslist.org/cto/d/round-rock...,austin,https://austin.craigslist.org,36000,1949.0,gmc,panel truck,NaN,NaN,...,NaN,NaN,NaN,https://images.craigslist.org/00P0P_aL9Sff00UG...,49 GMC PANEL TRUCK Custom-Classic 350 V8 aut...,NaN,tx,30.577676,-97.759094,2021-04-24T10:02:38-0500
131932,7314900957,https://boise.craigslist.org/ctd/d/nampa-2014-...,boise,https://boise.craigslist.org,18995,2014.0,ford,f150 supercrew cab lariat,excellent,6 cylinders,...,NaN,truck,NaN,https://images.craigslist.org/00I0I_ixEukvncXa...,2014 Ford F150 F 150 F-150 SuperCrew Cab Laria...,NaN,id,43.585854,-116.552067,2021-04-30T11:36:32-0600
146867,7316908875,https://stlouis.craigslist.org/ctd/d/fallon-20...,"st louis, MO",https://stlouis.craigslist.org,269,2014.0,ford,fusion se,NaN,4 cylinders,...,mid-size,sedan,silver,https://images.craigslist.org/00303_z6DuSjEkPH...,2014 Ford FUSION SE SE Auto Solutions Motor Co...,NaN,il,38.588255,-89.923709,2021-05-04T15:03:36-0500


In [15]:
data_train.isna()

,id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
366318,False,False,False,False,False,False,False,False,True,False,...,True,False,False,False,False,True,False,False,False,False
56271,False,False,False,False,False,False,False,False,True,False,...,True,False,True,False,False,True,False,False,False,False
264620,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
88787,False,False,False,False,False,False,True,False,True,True,...,True,False,False,False,False,True,False,False,False,False
341412,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,True,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
365838,False,False,False,False,False,False,False,False,True,True,...,True,True,True,False,False,True,False,False,False,False
131932,False,False,False,False,False,False,False,False,False,False,...,True,False,True,False,False,True,False,False,False,False
146867,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,True,False,False,False,False


In [16]:
data_train.isna().sum()

id                   0
url                  0
region               0
region_url           0
price                0
year               971
manufacturer     14062
model             4224
condition       139449
cylinders       142306
fuel              2375
odometer          3545
title_status      6554
transmission      2021
VIN             128743
drive           104481
size            244950
type             74395
paint_color     104288
image_url           55
description         57
county          341504
state                0
lat               5203
long              5203
posting_date        55
dtype: int64

In [17]:
data_train.isna().mean()

id              0.000000
url             0.000000
region          0.000000
region_url      0.000000
price           0.000000
year            0.002843
manufacturer    0.041177
model           0.012369
condition       0.408338
cylinders       0.416704
fuel            0.006955
odometer        0.010381
title_status    0.019192
transmission    0.005918
VIN             0.376988
drive           0.305944
size            0.717268
type            0.217845
paint_color     0.305379
image_url       0.000161
description     0.000167
county          1.000000
state           0.000000
lat             0.015236
long            0.015236
posting_date    0.000161
dtype: float64

In [18]:
data_train_v1 = data_train.drop(data_train.columns[data_train.isna().mean() > 0.25], axis = 1)

data_train_v1

,id,url,region,region_url,price,year,manufacturer,model,fuel,odometer,title_status,transmission,type,image_url,description,state,lat,long,posting_date
366318,7310804917,https://austin.craigslist.org/ctd/d/houston-20...,austin,https://austin.craigslist.org,46995,2017.0,ford,f-150,gas,48544.0,clean,automatic,truck,https://images.craigslist.org/00202_fA8s718RZU...,This 2017 Ford F-150 Lariat 4WD SuperCrew 5.5'...,tx,29.678380,-95.435120,2021-04-22T09:03:58-0500
56271,7315327041,https://sandiego.craigslist.org/nsd/ctd/d/palm...,san diego,https://sandiego.craigslist.org,0,2018.0,acura,mdx,gas,53103.0,clean,automatic,SUV,https://images.craigslist.org/00Q0Q_b7u8uPQBQE...,Coachella Valley Volkswagen STOCK #: P...,ca,33.755638,-116.286258,2021-05-01T08:55:38-0700
264620,7310280162,https://albany.craigslist.org/ctd/d/new-lebano...,albany,https://albany.craigslist.org,36900,2016.0,ford,f-250 super duty,gas,73394.0,clean,automatic,truck,https://images.craigslist.org/00c0c_jcqfcYXhpQ...,2016 Ford F-250 Super Duty XL 4x4 4dr Crew Cab...,ny,42.475900,-73.377300,2021-04-21T09:39:24-0400
88787,7315777281,https://washingtondc.craigslist.org/nva/ctd/d/...,"washington, DC",https://washingtondc.craigslist.org,1399,2021.0,NaN,WOLF ISLANDER,gas,500.0,clean,automatic,other,https://images.craigslist.org/00000_af225n2gi4...,2021 WOLF ISLANDER 50CC Offered by: Beyond...,dc,38.759303,-77.454234,2021-05-02T12:05:03-0400
341412,7306277173,https://york.craigslist.org/ctd/d/york-2018-bu...,york,https://york.craigslist.org,29590,2018.0,buick,enclave essence sport,gas,38459.0,clean,other,SUV,https://images.craigslist.org/00w0w_36PXCHco1K...,Carvana is the safer way to buy a car During t...,pa,39.960000,-76.730000,2021-04-13T14:21:01-0400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,7316779292,https://albuquerque.craigslist.org/ctd/d/albuq...,albuquerque,https://albuquerque.craigslist.org,6800,2013.0,volkswagen,jetta se,gas,139200.0,clean,manual,sedan,https://images.craigslist.org/01414_8DFTgDUqfH...,"2013 VW Jetta SE 2.5L 5-speed, A/C, pwr. windo...",nm,35.195300,-106.609300,2021-05-04T10:37:26-0600
365838,7311855271,https://austin.craigslist.org/cto/d/round-rock...,austin,https://austin.craigslist.org,36000,1949.0,gmc,panel truck,gas,27400.0,clean,automatic,NaN,https://images.craigslist.org/00P0P_aL9Sff00UG...,49 GMC PANEL TRUCK Custom-Classic 350 V8 aut...,tx,30.577676,-97.759094,2021-04-24T10:02:38-0500
131932,7314900957,https://boise.craigslist.org/ctd/d/nampa-2014-...,boise,https://boise.craigslist.org,18995,2014.0,ford,f150 supercrew cab lariat,gas,184147.0,NaN,other,truck,https://images.craigslist.org/00I0I_ixEukvncXa...,2014 Ford F150 F 150 F-150 SuperCrew Cab Laria...,id,43.585854,-116.552067,2021-04-30T11:36:32-0600
146867,7316908875,https://stlouis.craigslist.org/ctd/d/fallon-20...,"st louis, MO",https://stlouis.craigslist.org,269,2014.0,ford,fusion se,gas,111090.0,clean,automatic,sedan,https://images.craigslist.org/00303_z6DuSjEkPH...,2014 Ford FUSION SE SE Auto Solutions Motor Co...,il,38.588255,-89.923709,2021-05-04T15:03:36-0500


In [19]:
unneeded_columns = ['id', 'url', 'region_url', 'image_url', 'description', 'posting_date', 'model']

data_train_v2 = data_train_v1.drop(unneeded_columns, axis = 1)

data_train_v2

,region,price,year,manufacturer,fuel,odometer,title_status,transmission,type,state,lat,long
366318,austin,46995,2017.0,ford,gas,48544.0,clean,automatic,truck,tx,29.678380,-95.435120
56271,san diego,0,2018.0,acura,gas,53103.0,clean,automatic,SUV,ca,33.755638,-116.286258
264620,albany,36900,2016.0,ford,gas,73394.0,clean,automatic,truck,ny,42.475900,-73.377300
88787,"washington, DC",1399,2021.0,NaN,gas,500.0,clean,automatic,other,dc,38.759303,-77.454234
341412,york,29590,2018.0,buick,gas,38459.0,clean,other,SUV,pa,39.960000,-76.730000
...,...,...,...,...,...,...,...,...,...,...,...,...
259178,albuquerque,6800,2013.0,volkswagen,gas,139200.0,clean,manual,sedan,nm,35.195300,-106.609300
365838,austin,36000,1949.0,gmc,gas,27400.0,clean,automatic,NaN,tx,30.577676,-97.759094
131932,boise,18995,2014.0,ford,gas,184147.0,NaN,other,truck,id,43.585854,-116.552067
146867,"st louis, MO",269,2014.0,ford,gas,111090.0,clean,automatic,sedan,il,38.588255,-89.923709


In [20]:
data_train_v2

,region,price,year,manufacturer,fuel,odometer,title_status,transmission,type,state,lat,long
366318,austin,46995,2017.0,ford,gas,48544.0,clean,automatic,truck,tx,29.678380,-95.435120
56271,san diego,0,2018.0,acura,gas,53103.0,clean,automatic,SUV,ca,33.755638,-116.286258
264620,albany,36900,2016.0,ford,gas,73394.0,clean,automatic,truck,ny,42.475900,-73.377300
88787,"washington, DC",1399,2021.0,NaN,gas,500.0,clean,automatic,other,dc,38.759303,-77.454234
341412,york,29590,2018.0,buick,gas,38459.0,clean,other,SUV,pa,39.960000,-76.730000
...,...,...,...,...,...,...,...,...,...,...,...,...
259178,albuquerque,6800,2013.0,volkswagen,gas,139200.0,clean,manual,sedan,nm,35.195300,-106.609300
365838,austin,36000,1949.0,gmc,gas,27400.0,clean,automatic,NaN,tx,30.577676,-97.759094
131932,boise,18995,2014.0,ford,gas,184147.0,NaN,other,truck,id,43.585854,-116.552067
146867,"st louis, MO",269,2014.0,ford,gas,111090.0,clean,automatic,sedan,il,38.588255,-89.923709


In [27]:
{column: len(data[column].unique()) for column in data_v2.columns if data_v2.dtypes[column] == 'object'}

{'region': 404,
 'manufacturer': 43,
 'fuel': 6,
 'title_status': 7,
 'transmission': 4,
 'type': 14,
 'state': 51}

In [19]:
pd_d

,price,year,odometer,lat,long
0,6000.0,2011.235191,98043.331443,38.493940,-94.748599
1,11900.0,2011.235191,98043.331443,38.493940,-94.748599
2,21000.0,2011.235191,98043.331443,38.493940,-94.748599
3,1500.0,2011.235191,98043.331443,38.493940,-94.748599
4,4900.0,2011.235191,98043.331443,38.493940,-94.748599
...,...,...,...,...,...
426875,23590.0,2019.000000,32226.000000,33.786500,-84.445400
426876,30590.0,2020.000000,12029.000000,33.786500,-84.445400
426877,34990.0,2020.000000,4174.000000,33.779214,-84.411811
426878,28990.0,2018.000000,30112.000000,33.786500,-84.445400


In [21]:
def onehot_encode(df, columns, prefixes):
  df = df.copy()
  for column, prefix in zip(columns, prefixes):
    dummies = pd.get_dummies(df[column], prefix = prefix)
    df = pd.concat([df, dummies], axis = 1)
    df = df.drop(column, axis = 1)
  return df

In [22]:
data_train_v2_1h = onehot_encode(data_train_v2, ['region', 'manufacturer', 'fuel', 'title_status', 'transmission', 'type', 'state'], ['reg', 'manu', 'fuel', 'title', 'trans', 'type', 'state'])

In [24]:
data_train_v2_1h

,price,year,odometer,lat,long,reg_SF bay area,reg_abilene,reg_akron / canton,reg_albany,reg_albuquerque,...,state_sd,state_tn,state_tx,state_ut,state_va,state_vt,state_wa,state_wi,state_wv,state_wy
366318,46995,2017.0,48544.0,29.678380,-95.435120,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
56271,0,2018.0,53103.0,33.755638,-116.286258,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
264620,36900,2016.0,73394.0,42.475900,-73.377300,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
88787,1399,2021.0,500.0,38.759303,-77.454234,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
341412,29590,2018.0,38459.0,39.960000,-76.730000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259178,6800,2013.0,139200.0,35.195300,-106.609300,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
365838,36000,1949.0,27400.0,30.577676,-97.759094,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
131932,18995,2014.0,184147.0,43.585854,-116.552067,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
146867,269,2014.0,111090.0,38.588255,-89.923709,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [26]:
#Removing NaNs
for column in data_train_v2_1h.columns:
  data_train_v2_1h[column] = data_train_v2_1h[column].fillna(data_train_v2_1h[column].mean())

In [27]:
data_train_v2_1h.isna().sum().sum()

0

### Splitting and Scaling

In [28]:
y = data_train_v2_1h.loc[:, 'price']
X = data_train_v2_1h.drop('price', axis=1)

In [30]:
from sklearn.preprocessing import StandardScaler

In [31]:
scaler = StandardScaler()

In [32]:
X = scaler.fit_transform(X)

In [33]:
X

array([[ 0.61086993, -0.23488509, -1.51909441, ..., -0.16568195,
        -0.04977493, -0.03790635],
       [ 0.71671238, -0.2131675 , -0.81676343, ..., -0.16568195,
        -0.04977493, -0.03790635],
       [ 0.50502748, -0.11650778,  0.68535151, ..., -0.16568195,
        -0.04977493, -0.03790635],
       ...,
       [ 0.29334259,  0.41108349,  0.87654742, ..., -0.16568195,
        -0.04977493, -0.03790635],
       [ 0.29334259,  0.0630637 ,  0.01568245, ..., -0.16568195,
        -0.04977493, -0.03790635],
       [-0.02418475,  0.57552077, -0.80963876, ..., -0.16568195,
        -0.04977493, -0.03790635]])

In [37]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, random_state = 42)

### Training

In [34]:
from sklearn.linear_model import LinearRegression

In [35]:
linear_model = LinearRegression()

In [36]:
linear_model.fit(X, y)

LinearRegression()

In [40]:
data_test

data_test_v1 = data_test.drop(data_test.columns[data_test.isna().mean() > 0.25], axis = 1)

data_test_v2 = data_test_v1.drop(unneeded_columns, axis = 1)

data_test_v2

,region,price,year,manufacturer,fuel,odometer,title_status,transmission,type,state,lat,long
100905,lakeland,36990,2017.0,ford,gas,38094.0,clean,other,pickup,fl,28.040000,-81.960000
143835,"quad cities, IA/IL",27995,2006.0,chevrolet,gas,NaN,clean,manual,convertible,il,42.477800,-92.366100
20235,little rock,78423,2015.0,chevrolet,gas,30200.0,clean,automatic,convertible,ar,38.401800,-93.785000
300734,northern panhandle,14000,2013.0,bmw,gas,92965.0,clean,automatic,NaN,oh,40.320300,-80.625000
316249,eugene,676,2019.0,chevrolet,other,47105.0,clean,automatic,NaN,or,43.783900,-123.052900
...,...,...,...,...,...,...,...,...,...,...,...,...
285104,new hampshire,99,2018.0,jeep,other,44783.0,clean,automatic,NaN,nh,43.244000,-71.410500
204282,kalamazoo,6000,2015.0,chrysler,gas,104000.0,clean,automatic,NaN,mi,42.262400,-85.609600
418859,kenosha-racine,252,2018.0,nissan,gas,33827.0,NaN,automatic,sedan,wi,42.595596,-87.879408
93858,ft myers / SW florida,2650,2002.0,gmc,gas,140000.0,clean,automatic,NaN,fl,26.602457,-81.860569


In [71]:
data_test_v2_1h = onehot_encode(data_test_v2, ['region', 'manufacturer', 'fuel', 'title_status', 'transmission', 'type', 'state'], ['reg', 'manu', 'fuel', 'title', 'trans', 'type', 'state'])
data_test_v2_1h['temp'] = 0

In [72]:
for column in data_test_v2_1h.columns:
    data_test_v2_1h[column] = data_test_v2_1h[column].fillna(data_test_v2_1h[column].mean())

In [74]:
data_test_v2_1h

,price,year,odometer,lat,long,reg_SF bay area,reg_abilene,reg_akron / canton,reg_albany,reg_albuquerque,...,state_tn,state_tx,state_ut,state_va,state_vt,state_wa,state_wi,state_wv,state_wy,temp
100905,36990,2017.0,38094.000000,28.040000,-81.960000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
143835,27995,2006.0,98810.203405,42.477800,-92.366100,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20235,78423,2015.0,30200.000000,38.401800,-93.785000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
300734,14000,2013.0,92965.000000,40.320300,-80.625000,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
316249,676,2019.0,47105.000000,43.783900,-123.052900,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
285104,99,2018.0,44783.000000,43.244000,-71.410500,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
204282,6000,2015.0,104000.000000,42.262400,-85.609600,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
418859,252,2018.0,33827.000000,42.595596,-87.879408,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
93858,2650,2002.0,140000.000000,26.602457,-81.860569,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [75]:
data_test_v2_1h.isna().sum().sum()

0

In [76]:
y_test = data_test_v2_1h.loc[:, 'price']
X_test = data_test_v2_1h.drop('price', axis=1)

In [77]:
X_test = scaler.fit_transform(X_test)

X_test

array([[ 0.61032357, -0.27126348, -1.81224362, ..., -0.04941894,
        -0.037517  ,  0.        ],
       [-0.55968605,  0.        ,  0.69376261, ..., -0.04941894,
        -0.037517  ,  0.        ],
       [ 0.39759455, -0.30653172, -0.01371923, ..., -0.04941894,
        -0.037517  ,  0.        ],
       ...,
       [ 0.71668808, -0.29032727,  0.71420877, ..., -0.04941894,
        -0.037517  ,  0.        ],
       [-0.9851441 ,  0.18402481, -2.06176167, ..., -0.04941894,
        -0.037517  ,  0.        ],
       [ 0.71668808, -0.35791059, -1.47610296, ..., -0.04941894,
        -0.037517  ,  0.        ]])

In [78]:
y_preds = linear_model.predict(X_test)

### Using lightGBM for gradient-boosting

In [80]:
# import lightgbm as lgb  

ModuleNotFoundError: No module named 'lightgbm'

In [44]:
# lgb_model = lgb.LGBMRegressor(
#     boosting_type = 'gbdt',
#     reg_lambda = 1.0
# )
# lgb_model.fit(X_train, y_train)

# lgb_y_preds = lgb_model.predict(X_test)

In [82]:
from sklearn.metrics import mean_squared_error

In [83]:
lin_loss = np.sqrt(mean_squared_error(y_test, y_preds))
# lgb_loss = np.sqrt(mean_squared_error(y_test, lgb_y_preds))

In [84]:
print('Linear Regression RMSE: ', lin_loss)
# print('LGB RMSE: ', lgb_loss)

Linear Regression RMSE:  7.856785213524111e+18


In [87]:
print('Linear Regression R^2 Score: ', linear_model.score(X_test, y_test))
# print('GBM R^2 Score: ', lgb_model.score(X_test, y_test))

#Can be better!

Linear Regression R^2 Score:  -1.5768884173495227e+23


array([-1.79436844e+18, -1.19508917e+19,  2.58235500e+19, ...,
       -7.60666557e+18, -1.84463654e+18,  1.65889070e+18])


### Data set modifications using a pipeline

In [30]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin

In [31]:
# impute = SimpleImputer(strategy = 'mean')
# data_v2_num = data_v2.drop(['region', 'manufacturer', 'fuel', 'title_status', 'transmission', 'type', 'state'], axis = 1)

In [39]:
# encoder = OneHotEncoder()

In [69]:
# class DataFrameSelector(BaseEstimator, TransformerMixin):
#   def __init__(self, attribute_names):
#     self.attribute_names = attribute_names
#   def fit(self, X, y=None):
#     return self
#   def transform(self, X):
#     return X[self.attribute_names].values

In [109]:
data_v2

,region,price,year,manufacturer,fuel,odometer,title_status,transmission,type,state,lat,long
0,prescott,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN
1,fayetteville,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN
2,florida keys,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN
3,worcester / central MA,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN
4,greensboro,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
426875,wyoming,23590,2019.0,nissan,gas,32226.0,clean,other,sedan,wy,33.786500,-84.445400
426876,wyoming,30590,2020.0,volvo,gas,12029.0,clean,other,sedan,wy,33.786500,-84.445400
426877,wyoming,34990,2020.0,cadillac,diesel,4174.0,clean,other,hatchback,wy,33.779214,-84.411811
426878,wyoming,28990,2018.0,lexus,gas,30112.0,clean,other,sedan,wy,33.786500,-84.445400


In [110]:
data_v2_price = data_v2['price'].copy()

In [111]:
data_v2_price

0          6000
1         11900
2         21000
3          1500
4          4900
          ...  
426875    23590
426876    30590
426877    34990
426878    28990
426879    30590
Name: price, Length: 426880, dtype: int64

In [112]:
data_v2 = data_v2.drop('price', axis = 1)

In [113]:
data_v2

,region,year,manufacturer,fuel,odometer,title_status,transmission,type,state,lat,long
0,prescott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN
1,fayetteville,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN
2,florida keys,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN
3,worcester / central MA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN
4,greensboro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
426875,wyoming,2019.0,nissan,gas,32226.0,clean,other,sedan,wy,33.786500,-84.445400
426876,wyoming,2020.0,volvo,gas,12029.0,clean,other,sedan,wy,33.786500,-84.445400
426877,wyoming,2020.0,cadillac,diesel,4174.0,clean,other,hatchback,wy,33.779214,-84.411811
426878,wyoming,2018.0,lexus,gas,30112.0,clean,other,sedan,wy,33.786500,-84.445400


In [114]:
data_v2_cat = data_v2.drop(['year', 'odometer', 'lat', 'long'], axis = 1)
data_v2_cat

,region,manufacturer,fuel,title_status,transmission,type,state
0,prescott,NaN,NaN,NaN,NaN,NaN,az
1,fayetteville,NaN,NaN,NaN,NaN,NaN,ar
2,florida keys,NaN,NaN,NaN,NaN,NaN,fl
3,worcester / central MA,NaN,NaN,NaN,NaN,NaN,ma
4,greensboro,NaN,NaN,NaN,NaN,NaN,nc
...,...,...,...,...,...,...,...
426875,wyoming,nissan,gas,clean,other,sedan,wy
426876,wyoming,volvo,gas,clean,other,sedan,wy
426877,wyoming,cadillac,diesel,clean,other,hatchback,wy
426878,wyoming,lexus,gas,clean,other,sedan,wy


In [131]:
num_attribs = ['year', 'odometer', 'lat', 'long']
num_attribs

['year', 'odometer', 'lat', 'long']

In [132]:
data_v2_num = data_v2[num_attribs].copy()

In [133]:
data_v2_num

,year,odometer,lat,long
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN
...,...,...,...,...
426875,2019.0,32226.0,33.786500,-84.445400
426876,2020.0,12029.0,33.786500,-84.445400
426877,2020.0,4174.0,33.779214,-84.411811
426878,2018.0,30112.0,33.786500,-84.445400


In [134]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [135]:

cat_attribs = ['region', 'manufacturer', 'fuel', 'title_status', 'transmission', 'type', 'state']

num_pipeline = Pipeline([
              ('impute', SimpleImputer(strategy = 'mean')),
              ('std_scaler', StandardScaler()),
])
cat_pipeline = Pipeline([
              ('selector', DataFrameSelector(cat_attribs)),
              ('cat_encoder', OneHotEncoder(categories=cat_attribs)),
])

In [136]:
cat_attribs

['region',
 'manufacturer',
 'fuel',
 'title_status',
 'transmission',
 'type',
 'state']

In [137]:
(num_attribs,
 cat_attribs)

(['year', 'odometer', 'lat', 'long'],
 ['region',
  'manufacturer',
  'fuel',
  'title_status',
  'transmission',
  'type',
  'state'])

In [138]:
oHe = OneHotEncoder()

In [139]:
full_pipeline = ColumnTransformer([
                                               ('num_pipeline', num_pipeline, num_attribs),
                                               ('cat_pipeline', OneHotEncoder(), cat_attribs),

                                               
])

In [140]:
data_v2

,region,year,manufacturer,fuel,odometer,title_status,transmission,type,state,lat,long
0,prescott,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN
1,fayetteville,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN
2,florida keys,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN
3,worcester / central MA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN
4,greensboro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
426875,wyoming,2019.0,nissan,gas,32226.0,clean,other,sedan,wy,33.786500,-84.445400
426876,wyoming,2020.0,volvo,gas,12029.0,clean,other,sedan,wy,33.786500,-84.445400
426877,wyoming,2020.0,cadillac,diesel,4174.0,clean,other,hatchback,wy,33.779214,-84.411811
426878,wyoming,2018.0,lexus,gas,30112.0,clean,other,sedan,wy,33.786500,-84.445400


In [141]:
dataset_prepared = full_pipeline.fit_transform(data_v2)

In [142]:
dataset_prepared

<426880x533 sparse matrix of type '<class 'numpy.float64'>'
	with 4684731 stored elements in Compressed Sparse Row format>

In [143]:
import scipy.sparse

In [144]:
data_v2_prepared = pd.DataFrame.sparse.from_spmatrix(dataset_prepared)

In [145]:
data_v2_prepared

,0,1,2,3,4,5,6,7,8,9,...,523,524,525,526,527,528,529,530,531,532
0,-2.408936e-14,0.000000,0.000000,2.339361e-15,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-2.408936e-14,0.000000,0.000000,2.339361e-15,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-2.408936e-14,0.000000,0.000000,2.339361e-15,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-2.408936e-14,0.000000,0.000000,2.339361e-15,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-2.408936e-14,0.000000,0.000000,2.339361e-15,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
426875,8.226515e-01,-0.309327,-0.812111,5.653637e-01,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
426876,9.285977e-01,-0.404248,-0.812111,5.653637e-01,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
426877,9.285977e-01,-0.441165,-0.813368,5.672068e-01,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
426878,7.167054e-01,-0.319262,-0.812111,5.653637e-01,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [146]:
data_v2_prepared.isna().sum().sum()

0

In [152]:
from sklearn.linear_model import LinearRegression
lin_reg_v3 = LinearRegression()

In [157]:
scaler = StandardScaler()

In [153]:
data_v2_price

0          6000
1         11900
2         21000
3          1500
4          4900
          ...  
426875    23590
426876    30590
426877    34990
426878    28990
426879    30590
Name: price, Length: 426880, dtype: int64

In [ ]:
data_v2_price_prepared = num_pipeline.fit_transform(data_v2_price)


In [149]:
lin_reg_v2.fit(data_v2_prepared, data_v2_price_prepared)

LinearRegression()

In [151]:
some_data = data_v2.iloc[:5]
some_labels = data_v2_price.iloc[:5]
some_data_prepared = full_pipeline.transform(some_data)
print("Predictions:", lin_reg_v2.predict(some_data_prepared))

print("Labels:", list(some_labels))


Predictions: [-42258.27599039 -53126.53375231  43842.08447351 -30089.10113984
 -49700.15506704]
Labels: [6000, 11900, 21000, 1500, 4900]
